In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

from demres.common.helper_functions import *
from demres.common.process_entries import *

In [16]:
pd.options.display.max_rows = 5000
pd.set_option('display.max_columns', None)

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [6]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins.csv',delimiter=',')

## Explore patient history

In [9]:
all_entries = get_all_entries()

In [10]:
hist = get_patient_history(all_entries,7457038).sort_values(by='eventdate')

In [11]:
pt_details = pt_features[pt_features['patid']==7457038]

In [54]:
get_windows()

[{'end': datetime.timedelta(1825), 'start': datetime.timedelta(3650)},
 {'end': datetime.timedelta(0), 'start': datetime.timedelta(1825)}]

In [53]:
pt_details.T

,11705
patid,7457038
gender,1
yob,1930
pracid,38
index_date,2008-08-05
isCase,False
final dementia medcode,0
data_start,1995-07-30
data_end,2010-10-01
matchid,5612


In [51]:
# matching_pt = pt_features[(pt_features['matchid']==pt_details['matchid'].iloc[0])&(pt_features['patid']!=pt_details['patid'].iloc[0])]
# matching_pt

In [18]:
index_date = pd.to_datetime(pt_details['index_date']).iloc[0]

In [38]:
# hist.sort_values(by='eventdate')

In [32]:
windows = get_windows()
windows

[{'end': datetime.timedelta(1825), 'start': datetime.timedelta(3650)},
 {'end': datetime.timedelta(0), 'start': datetime.timedelta(1825)}]

In [30]:
window0 = hist[(hist['eventdate']>(index_date-windows[0]['start']))&(hist['eventdate']<(index_date-windows[0]['end']))]

In [31]:
window0.sort_values(by='eventdate')

,patid,eventdate,sysdate,type,description
1487,7457038,1998-09-07,1998-09-07,prescription,Cefuroxime axetil
1486,7457038,1998-09-07,1998-09-07,prescription,Acetic acid glacial/Dexamethasone/Neomycin sul...
154,7457038,1998-09-07,1998-09-07,consultation,NaN
3021,7457038,1998-09-07,1998-09-07,clinical,Chest infection
3020,7457038,1998-09-07,1998-09-07,clinical,Otitis externa NOS
155,7457038,1998-09-17,1998-09-17,consultation,NaN
1489,7457038,1998-09-17,1998-09-17,prescription,Beclometasone dipropionate
1488,7457038,1998-09-17,1998-09-17,prescription,Salbutamol
3022,7457038,1998-09-28,1998-09-28,clinical,Chest infection
3024,7457038,1998-09-28,1998-09-28,clinical,Hearing difficulty


## Explore patient prescriptions

In [ ]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [ ]:
prescriptions.head(1)

In [ ]:
pegprod = pd.read_csv('data/dicts/proc_pegasus_prod.csv')
prescs = pd.merge(prescriptions,pegprod[['prodcode','route','substance strength','drug substance name']],how='left')

In [ ]:
prescs = pd.merge(prescs,pt_features[['patid','index_date']],how='left')

In [ ]:
prescs.drop(['numdays','type','numpacks','packtype','issueseq'],axis=1,inplace=True)

In [ ]:
pt_prescs = prescs[prescs['patid']==8146013].sort_values(by='eventdate')

In [ ]:
pt_details = pt_features[pt_features['patid']==8146013]

In [ ]:
index_date = pd.to_datetime(pt_details['index_date']).iloc[0]

In [ ]:
index_date

In [ ]:
pt_prescs[(pt_prescs['eventdate']>(index_date-timedelta(days=365)*10))&(pt_prescs['eventdate']<(index_date-timedelta(days=365)*5))]